In [3]:
import pandas as pd 
import numpy as np 
import pickle 

In [32]:
train = pd.read_csv('train.csv')

In [4]:
transaction = pd.read_csv('transactions.csv')

In [5]:
member = pd.read_csv('members_v3.csv')

In [35]:
user_logs = pd.read_csv('user_logs.csv',nrows=20000000)

### Extracting features from Transaction 

In [7]:
transaction.head(1)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0


In [8]:
## Discount Feature 

transaction['discount']= transaction['plan_list_price']-transaction['actual_amount_paid']

In [9]:
transaction.head(1)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0,0


In [10]:
## is_discount : To check weather the discount is given to customer or not 

transaction['is_discount'] = transaction.discount.apply(lambda x:1 if x>0 else 0)
transaction.head(3)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount,is_discount
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0,0,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0,0,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0,0,0


In [11]:
### Membership Duration of a customer 

transaction['membership_duration'] = transaction.membership_expire_date - transaction.transaction_date
transaction.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount,is_discount,membership_duration
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0,0,0,171
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0,0,0,101
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0,0,0,9497
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0,0,0,198
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0,0,0,191


In [12]:
transaction_members = pd.merge(transaction,member,on='msno',how='inner')

In [13]:
## Free space 
del transaction 
del member 

In [17]:
transaction_members.head(1)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount,is_discount,membership_duration,city,bd,gender,registered_via,registration_init_time
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0,0,0,171,1,0,NaN,7,20110629


In [23]:
## Customer who have auto-renewed and not cancelled 

transaction_members['auto-renew_not_cancel'] = ((transaction_members.is_auto_renew==1)==(transaction_members.is_cancel==0)).astype(np.int8)


In [24]:
transaction_members.head(2)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount,is_discount,membership_duration,city,bd,gender,registered_via,registration_init_time,auto-renew_not_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0,0,0,171,1,0,NaN,7,20110629,1
1,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20151031,20151201,0,0,0,170,1,0,NaN,7,20110629,1


In [25]:
## Customer who have not auto_renew_cancel 

transaction_members['not_auto-renew_cancel'] = ((transaction_members.is_auto_renew==0)==(transaction_members.is_cancel==1)).astype(np.int8)


In [31]:
transaction_members.head(-1)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount,is_discount,membership_duration,city,bd,gender,registered_via,registration_init_time,auto-renew_not_cancel,not_auto-renew_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0,0,0,171,1,0,NaN,7,20110629,1,1
1,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20151031,20151201,0,0,0,170,1,0,NaN,7,20110629,1,1
2,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150630,20150801,0,0,0,171,1,0,NaN,7,20110629,1,1
3,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150228,20150401,0,0,0,173,1,0,NaN,7,20110629,1,1
4,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20161130,20170101,0,0,0,8971,1,0,NaN,7,20110629,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18891697,IeDvCreWzlgO8lqKCL8k1+QCvbCRAHFK/jAhtxK9uw4=,35,7,0,0,0,20161029,20161105,0,0,0,76,1,29,male,4,20160625,0,0
18891698,ON/0c5eeg3wKmtjyvjvCrza8959BpFO0T+5BwP5+duI=,22,395,1599,1599,0,20150818,20160921,0,0,0,10103,6,30,male,9,20060305,0,0
18891699,jQl+rtRwQMvHJc4jWO+Mj5f5EDvz+m04x7ASAFQ+pzo=,35,7,0,0,0,20161031,20161107,0,0,0,76,5,32,female,3,20150818,0,0
18891700,mvE2SExeEtdT1RTspjWkjiVuZ+0uQUn3+rC7Hdv39p0=,17,195,0,0,0,20150111,20150114,0,0,0,3,13,22,male,3,20150104,0,0


In [29]:
transaction_members.to_pickle('transaction_member.pkl')

In [4]:
transaction_members = pd.read_pickle('transaction_member.pkl')

In [5]:
## There were some cases where discount was negative 
transaction_members = transaction_members[transaction_members['discount']>0]


In [6]:
transaction_members.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount,is_discount,membership_duration,city,bd,gender,registered_via,registration_init_time,auto-renew_not_cancel,not_auto-renew_cancel
815,o/7kLhrPCLGM9bhiD72KnzyESFAA8e0j0hQ5E0ZV6Uk=,23,30,149,129,1,20150630,20150630,0,20,1,0,1,0,male,3,20150619,1,1
1032,HO1MPlRi0XgbS9vf57gWXWov+eaUdBIWu0KNHGsV1fM=,41,30,149,119,1,20150331,20150430,0,30,1,99,1,0,NaN,7,20140701,1,1
1033,HO1MPlRi0XgbS9vf57gWXWov+eaUdBIWu0KNHGsV1fM=,41,30,149,119,1,20150131,20150228,0,30,1,97,1,0,NaN,7,20140701,1,1
1035,HO1MPlRi0XgbS9vf57gWXWov+eaUdBIWu0KNHGsV1fM=,41,30,149,119,1,20150531,20150630,0,30,1,99,1,0,NaN,7,20140701,1,1
1040,HO1MPlRi0XgbS9vf57gWXWov+eaUdBIWu0KNHGsV1fM=,41,30,149,119,1,20150228,20150331,0,30,1,103,1,0,NaN,7,20140701,1,1


In [14]:
## Finding the counts of How many times a we have a particular customer imformation 
counts_transaction =transaction_members.groupby(['msno'], as_index=False)['actual_amount_paid','payment_plan_days','discount'].count()
 

C:\Users\Admin\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [15]:
counts_transaction.head(2)

,msno,actual_amount_paid,payment_plan_days,discount
0,++/gTmVgKUbNFmsTiriZdWV1uZIrLXCUiEWN0fEU6BM=,1,1,1
1,++3A6JMzYJeron30GTcDostfXoAl8rTBuB2M8GeVdNU=,6,6,6


In [9]:
## Finding the sum for finding the Average Later 
sum_transaction = transaction_members.groupby(['msno'],as_index=False)['actual_amount_paid','payment_plan_days','discount'].sum()


C:\Users\Admin\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [10]:
sum_transaction.head(2)

,msno,actual_amount_paid,payment_plan_days,discount
0,++/gTmVgKUbNFmsTiriZdWV1uZIrLXCUiEWN0fEU6BM=,0,30,149
1,++3A6JMzYJeron30GTcDostfXoAl8rTBuB2M8GeVdNU=,714,180,180


In [16]:
counts_transaction[['actual_amount_paid_avg','payment_plan_days_avg','discount_avg']] = sum_transaction[['actual_amount_paid','payment_plan_days','discount']]/counts_transaction[['actual_amount_paid','payment_plan_days','discount']]

In [17]:
counts_transaction.head()

,msno,actual_amount_paid,payment_plan_days,discount,actual_amount_paid_avg,payment_plan_days_avg,discount_avg
0,++/gTmVgKUbNFmsTiriZdWV1uZIrLXCUiEWN0fEU6BM=,1,1,1,0.0,30.0,149.0
1,++3A6JMzYJeron30GTcDostfXoAl8rTBuB2M8GeVdNU=,6,6,6,119.0,30.0,30.0
2,++8q+lB3TM+BUousCkspFgXvR0y2MQnxqCJqd60OcjA=,1,1,1,0.0,30.0,149.0
3,++92FghbCQPqDmQ96QzNiuEMoDxrMOmuaisu1UCrYn0=,4,4,4,129.0,30.0,20.0
4,++A8p4GrsTnMjI6hAZEtlRsaz6s6O9ddUoH0fmS4s7s=,2,2,2,0.0,30.0,149.0


In [18]:
sum_transaction = sum_transaction.merge(counts_transaction,on='msno')

In [19]:
sum_transaction.head()

,msno,actual_amount_paid_x,payment_plan_days_x,discount_x,actual_amount_paid_y,payment_plan_days_y,discount_y,actual_amount_paid_avg,payment_plan_days_avg,discount_avg
0,++/gTmVgKUbNFmsTiriZdWV1uZIrLXCUiEWN0fEU6BM=,0,30,149,1,1,1,0.0,30.0,149.0
1,++3A6JMzYJeron30GTcDostfXoAl8rTBuB2M8GeVdNU=,714,180,180,6,6,6,119.0,30.0,30.0
2,++8q+lB3TM+BUousCkspFgXvR0y2MQnxqCJqd60OcjA=,0,30,149,1,1,1,0.0,30.0,149.0
3,++92FghbCQPqDmQ96QzNiuEMoDxrMOmuaisu1UCrYn0=,516,120,80,4,4,4,129.0,30.0,20.0
4,++A8p4GrsTnMjI6hAZEtlRsaz6s6O9ddUoH0fmS4s7s=,0,60,298,2,2,2,0.0,30.0,149.0


In [20]:
sum_average_transaction = sum_transaction[['msno','actual_amount_paid_y','payment_plan_days_y','discount_y']]

In [21]:
transaction_members = transaction_members.merge(sum_average_transaction,on='msno')

In [27]:
transaction_members.head(100)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount,...,city,bd,gender,registered_via,registration_init_time,auto-renew_not_cancel,not_auto-renew_cancel,actual_amount_paid_y,payment_plan_days_y,discount_y
0,o/7kLhrPCLGM9bhiD72KnzyESFAA8e0j0hQ5E0ZV6Uk=,23,30,149,129,1,20150630,20150630,0,20,...,1,0,male,3,20150619,1,1,1,1,1
1,HO1MPlRi0XgbS9vf57gWXWov+eaUdBIWu0KNHGsV1fM=,41,30,149,119,1,20150331,20150430,0,30,...,1,0,NaN,7,20140701,1,1,4,4,4
2,HO1MPlRi0XgbS9vf57gWXWov+eaUdBIWu0KNHGsV1fM=,41,30,149,119,1,20150131,20150228,0,30,...,1,0,NaN,7,20140701,1,1,4,4,4
3,HO1MPlRi0XgbS9vf57gWXWov+eaUdBIWu0KNHGsV1fM=,41,30,149,119,1,20150531,20150630,0,30,...,1,0,NaN,7,20140701,1,1,4,4,4
4,HO1MPlRi0XgbS9vf57gWXWov+eaUdBIWu0KNHGsV1fM=,41,30,149,119,1,20150228,20150331,0,30,...,1,0,NaN,7,20140701,1,1,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,TRTLeqIakDjx6MVzcINNWjFjGL/3+yODk4SoRrxDi44=,41,30,149,119,1,20150531,20160502,0,30,...,1,0,NaN,7,20140429,1,1,5,5,5
96,TRTLeqIakDjx6MVzcINNWjFjGL/3+yODk4SoRrxDi44=,41,30,149,119,1,20150630,20160702,0,30,...,1,0,NaN,7,20140429,1,1,5,5,5
97,TRTLeqIakDjx6MVzcINNWjFjGL/3+yODk4SoRrxDi44=,41,30,149,119,1,20150331,20160201,0,30,...,1,0,NaN,7,20140429,1,1,5,5,5
98,TRTLeqIakDjx6MVzcINNWjFjGL/3+yODk4SoRrxDi44=,41,30,149,119,1,20150131,20151004,0,30,...,1,0,NaN,7,20140429,1,1,5,5,5


In [24]:
transaction_members.columns

Index(['msno', 'payment_method_id', 'payment_plan_days', 'plan_list_price',
       'actual_amount_paid', 'is_auto_renew', 'transaction_date',
       'membership_expire_date', 'is_cancel', 'discount', 'is_discount',
       'membership_duration', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'auto-renew_not_cancel',
       'not_auto-renew_cancel', 'actual_amount_paid_y', 'payment_plan_days_y',
       'discount_y'],
      dtype='object')

In [25]:
transaction_members.to_pickle('transaction_members.pkl')

##  User_Logs 

In [36]:
user_logs_train = pd.merge(train['msno'],user_logs,on='msno',how='left')

In [38]:
user_logs_train.shape

(12733908, 9)

#### So Here we found that 8M customer id did'nt match 

In [39]:
### As we are  only taking data before the 2016 dec 

## taking all date values before 31 december 2016

user_logs_train = user_logs_train[user_logs_train['date'].le(20161231)]

In [40]:
user_logs_train.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
3,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,20150122.0,0.0,1.0,0.0,0.0,12.0,12.0,3044.816
4,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,20150314.0,0.0,0.0,0.0,1.0,83.0,26.0,23020.346
5,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,20150509.0,0.0,0.0,0.0,2.0,34.0,24.0,8874.010
6,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,20150704.0,0.0,0.0,0.0,0.0,15.0,10.0,4036.832
7,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,20150828.0,0.0,0.0,0.0,0.0,5.0,5.0,1391.616


### Finding all the Sum Features 

In [46]:
user_log_sum_25 = user_logs_train.groupby(['msno'])['num_25'].sum()
user_log_sum_50 = user_logs_train.groupby(['msno'])['num_50'].sum()
user_log_sum_75 = user_logs_train.groupby(['msno'])['num_75'].sum()
user_log_sum_985 = user_logs_train.groupby(['msno'])['num_985'].sum()
user_log_sum_100 = user_logs_train.groupby(['msno'])['num_100'].sum()
user_log_sum_unq = user_logs_train.groupby(['msno'])['num_unq'].sum()
user_log_sum_total_sec = user_logs_train.groupby(['msno'])['total_secs'].sum()

In [57]:
user_log_sum_25.to_csv('user_log_sum_25.csv')
user_log_sum_50.to_csv('user_log_sum_50.csv')
user_log_sum_75.to_csv('user_log_sum_75.csv')
user_log_sum_985.to_csv('user_log_sum_985.csv')
user_log_sum_100.to_csv('user_log_sum_100.csv')
user_log_sum_unq.to_csv('user_log_sum_unq.csv')
user_log_sum_total_sec.to_csv('user_log_sum_total_sec.csv')

### Finding all the Count Features 

In [51]:
## As count will be same for all the features ,Did for 1 feature only 
user_log_count = user_logs_train.groupby(['msno'])['num_25'].count()

In [52]:
user_log_count.to_csv('user_log_count.csv')

### Finding the Average of the Features using sum/count features 

In [55]:
user_log_avg_25 = user_log_sum_25/user_log_count
user_log_avg_50 = user_log_sum_25/user_log_count
user_log_avg_75 = user_log_sum_25/user_log_count
user_log_avg_985 = user_log_sum_25/user_log_count
user_log_avg_100 = user_log_sum_25/user_log_count
user_log_avg_unq = user_log_sum_25/user_log_count
user_log_avg_total_sec = user_log_sum_25/user_log_count

In [56]:
user_log_avg_25.to_csv('user_log_avg_25.csv')
user_log_avg_50.to_csv('user_log_avg_50.csv')
user_log_avg_75.to_csv('user_log_avg_75.csv')
user_log_avg_985.to_csv('user_log_avg_985.csv')
user_log_avg_100.to_csv('user_log_avg_100.csv')
user_log_avg_unq.to_csv('user_log_avg_unq.csv')
user_log_avg_total_sec.to_csv('user_log_avg_total_sec.csv')